In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
import pandas as pd

import constants
import data_api

In [6]:
df = data_api.load_data()

In [7]:
# date_fields = ['closed_date', 'created_date', 'due_date']
# df_orig = pd.read_csv('data/2017.csv', index_col=0, parse_dates=date_fields)
# df = df_orig
df.head()

,address_type,agency,agency_name,borough,bridge_highway_segment,city,closed_date,community_board,complaint_type,created_date,...,school_number,school_phone_number,school_region,school_state,school_zip,status,street_name,unique_key,x_coordinate_state_plane,y_coordinate_state_plane
0,NaN,DPR,Department of Parks and Recreation,BRONX,NaN,BRONX,NaT,07 BRONX,Maintenance or Facility,2017-04-12 00:00:18,...,X013,7184301800,NaN,NY,10468,Open,NaN,35926527,NaN,NaN
1,ADDRESS,NYPD,New York City Police Department,MANHATTAN,NaN,NEW YORK,NaT,04 MANHATTAN,Noise - Residential,2017-04-12 00:00:18,...,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Open,WEST 46 STREET,35923369,987400.0,216201.0
2,ADDRESS,NYPD,New York City Police Department,BROOKLYN,NaN,BROOKLYN,2017-04-12 01:30:52,12 BROOKLYN,Noise - Residential,2017-04-12 00:00:33,...,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Closed,58 STREET,35921287,982765.0,170559.0
3,ADDRESS,NYPD,New York City Police Department,MANHATTAN,NaN,NEW YORK,NaT,09 MANHATTAN,Noise - Street/Sidewalk,2017-04-12 00:00:45,...,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Assigned,WEST 141 STREET,35925662,997333.0,239507.0
4,ADDRESS,NYPD,New York City Police Department,BRONX,NaN,BRONX,NaT,01 BRONX,Noise - Street/Sidewalk,2017-04-12 00:01:23,...,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Open,PROSPECT AVENUE,35925657,1010938.0,236138.0


In [ ]:
df_orig.incident_zip.describe()

In [ ]:
agency_dummies = pd.get_dummies(df[constants.AGENCY])
# agency_dummies.head()
df = pd.concat([df, agency_dummies], axis=1)
df.head()

In [ ]:
# Rules
# xxxxx-xxxx --> int(xxxxx)
# strings --> null
# Float --> int
# fill left with 0's to len=5
def clean_zips(zip):
    zip = str(zip).replace('.0', '')[:5]
    try:
        zip = int(zip)
    except ValueError:
        return None
    return '{:05}'.format(zip)
    
df['incident_zip'] = df.incident_zip.apply(clean_zips)

# for zip in df.incident_zip.unique():
#     print(zip)

In [ ]:
pd.get_dummies(df['incident_zip'].astype(str), prefix='zip').head()

In [ ]:
zip_dummies = pd.get_dummies(df['incident_zip'], prefix='zip')
df = pd.concat([df, zip_dummies], axis=1)
df.head()

In [ ]:
# Generate dummies for time of day
df['created_hr_of_day'] = df['created_date'].dt.hour
hour_of_day_dummies = pd.get_dummies(df['created_hr_of_day'])
df = pd.concat([df, hour_of_day_dummies], axis=1)
df.head()

In [ ]:
# Generate dummies for created_date - weekday_name
df['created_day_of_week'] = df['created_date'].dt.weekday_name
day_of_week_dummies = pd.get_dummies(df['created_day_of_week'])
df = pd.concat([df, day_of_week_dummies], axis=1)
df.head()

In [ ]:
# Generate dummies for created_date - month
df['created_month'] = df['created_date'].dt.month
month_dummies = pd.get_dummies(df['created_month'], prefix='month')
df = pd.concat([df, month_dummies], axis=1)
df.head()

In [ ]:
# Generate dummies for complaint_type
complaint_type_dummies = pd.get_dummies(df['complaint_type'], prefix='complaint_type')
df = pd.concat([df, complaint_type_dummies], axis=1)
df.head()

In [ ]:
# Generate dummies for community_board
community_board_dummies = pd.get_dummies(df['community_board'], prefix='community_board')
df = pd.concat([df, community_board_dummies], axis=1)
df.head()

In [ ]:
# Drop NA for closed_date
df.dropna(subset=['closed_date'], inplace=True)

In [ ]:
df = df.drop(df[df.closed_date - df.created_date < pd.Timedelta(0)].index)

In [ ]:
target_hours = (df.closed_date - df.created_date) / pd.Timedelta(hours=1)
target_hours.head()

In [ ]:
bin_mapping = {
    1: '< 1 hr',
    3: '1-3 hrs',
    6: '3-6 hrs',
    12: '6-12 hrs',
    24: '12-24 hrs', 
    36: '24-36 hrs',
    48: '36-48 hrs',
    7*24: '2-7 days',
    28*24: '1-4 weeks',
    100000: '> 1 month',
}
bins = [0] + list(bin_mapping.keys())
labels = list(bin_mapping.values())
target_categorized = pd.cut(target_hours, bins=bins, labels=labels)

In [ ]:
target = target_categorized

In [ ]:
target.value_counts()

In [ ]:
pd.set_option('display.max_columns', 1000)
# pd.set_option('display.max_columns', 30)

In [ ]:
df[target / pd.Timedelta(minutes=1) < 2].complaint_type.value_counts()

In [ ]:
# df[target / pd.Timedelta(minutes=1) < 2][df.complaint_type == 'Street Light Condition'].resolution_description.value_counts()
df[target / pd.Timedelta(minutes=1) > -1][df.complaint_type == 'Adopt-A-Basket'].resolution_description.value_counts()

In [ ]:
# < 1, 1-3, 3-6, 6-12, 0.5, 1, 1.5, 2, 3, 7, 30+


In [ ]:
# (target / pd.Timedelta(days=1)).hist(bins=range(0, 100))
# (target / pd.Timedelta(hours=1)).hist(bins=range(240, 1000, 24))
(target / pd.Timedelta(hours=1)).hist(bins=[0, 1, 3, 6, 12, 24, 36, 48, 72, 168, 720])

In [ ]:
target.describe()

In [ ]:
df.drop('closed_date', axis=1, inplace=True)

In [ ]:
# pd.set_option('display.max_columns', 1000)
# df.loc[df['complaint_type'] == 'Harboring Bees/Wasps']
# pd.set_option('display.max_columns', 30)

In [ ]:
df.info(verbose=True)
# df.info(ver)

In [21]:
import data_api
import transformers
from datetime import datetime

In [ ]:
df_file_path = data_api.load_recent_data()

In [54]:
print(df_file_path)

data/2017-01-01-2017-04-13.csv


In [55]:

df = data_api.load_data(filepath=df_file_path)
df.info()

/Users/Alexandra/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (18,20,22,25,50) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 661940 entries, 0 to 661939
Data columns (total 52 columns):
address_type                      629720 non-null object
agency                            661940 non-null object
agency_name                       661940 non-null object
borough                           661940 non-null object
bridge_highway_direction          1537 non-null object
bridge_highway_name               1539 non-null object
bridge_highway_segment            2643 non-null object
city                              617041 non-null object
closed_date                       602463 non-null datetime64[ns]
community_board                   661940 non-null object
complaint_type                    661940 non-null object
created_date                      661940 non-null datetime64[ns]
cross_street_1                    306412 non-null object
cross_street_2                    304717 non-null object
descriptor                        652753 non-null object
due_date                

In [58]:
(X, y) = transformers.get_sample_target_data(df)
X.head()

,address_type,agency_name,borough,bridge_highway_direction,bridge_highway_name,bridge_highway_segment,city,cross_street_1,cross_street_2,descriptor,...,community_board_81 QUEENS,community_board_82 QUEENS,community_board_83 QUEENS,community_board_84 QUEENS,community_board_95 STATEN ISLAND,community_board_Unspecified BRONX,community_board_Unspecified BROOKLYN,community_board_Unspecified MANHATTAN,community_board_Unspecified QUEENS,community_board_Unspecified STATEN ISLAND
0,ADDRESS,Department of Housing Preservation and Develop...,BRONX,NaN,NaN,NaN,BRONX,NaN,NaN,GARBAGE/RECYCLING STORAGE,...,0,0,0,0,0,0,0,0,0,0
1,ADDRESS,Department of Housing Preservation and Develop...,BROOKLYN,NaN,NaN,NaN,BROOKLYN,NaN,NaN,SEWAGE,...,0,0,0,0,0,0,0,0,0,0
2,ADDRESS,Department of Housing Preservation and Develop...,BROOKLYN,NaN,NaN,NaN,BROOKLYN,NaN,NaN,DOOR,...,0,0,0,0,0,0,0,0,0,0
3,ADDRESS,Department of Housing Preservation and Develop...,QUEENS,NaN,NaN,NaN,Jamaica,NaN,NaN,DOOR,...,0,0,0,0,0,0,0,0,0,0
4,ADDRESS,Department of Health and Mental Hygiene,BRONX,NaN,NaN,NaN,BRONX,WEST KINGSBRIDGE ROAD,EAMES PLACE,Chemical Vapors/Gases/Odors,...,0,0,0,0,0,0,0,0,0,0


In [35]:
y.value_counts()

2 - 7 days     135643
1 - 3 hrs       76311
1 - 4 weeks     67911
3 - 6 hrs       54435
12 - 24 hrs     54394
< 1 hr          52663
24 - 36 hrs     37912
6 - 12 hrs      37459
36 - 48 hrs     29307
> 4 weeks       20810
dtype: int64

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

In [59]:
# 75/25 training/test split
X_train, X_test, y_train, y_test = train_test_split(X, y)
clf = SGDClassifier(loss="hinge", penalty="l2")
clf.fit(X_train, y_train)

ValueError: could not convert string to float: 'Closed'

In [40]:
clf.score(X_test, y_test)

,address_type,agency,agency_name,borough,bridge_highway_direction,bridge_highway_name,bridge_highway_segment,city,community_board,complaint_type,...,community_board_81 QUEENS,community_board_82 QUEENS,community_board_83 QUEENS,community_board_84 QUEENS,community_board_95 STATEN ISLAND,community_board_Unspecified BRONX,community_board_Unspecified BROOKLYN,community_board_Unspecified MANHATTAN,community_board_Unspecified QUEENS,community_board_Unspecified STATEN ISLAND
311335,ADDRESS,HPD,Department of Housing Preservation and Develop...,BRONX,NaN,NaN,NaN,BRONX,09 BRONX,HEAT/HOT WATER,...,0,0,0,0,0,0,0,0,0,0
302746,BLOCKFACE,DOT,Department of Transportation,QUEENS,NaN,NaN,NaN,Jamaica,12 QUEENS,Street Condition,...,0,0,0,0,0,0,0,0,0,0
602670,ADDRESS,HPD,Department of Housing Preservation and Develop...,BROOKLYN,NaN,NaN,NaN,BROOKLYN,14 BROOKLYN,PLUMBING,...,0,0,0,0,0,0,0,0,0,0
414136,ADDRESS,DSNY,BCC - Queens East,QUEENS,NaN,NaN,NaN,Oakland Gardens,11 QUEENS,Request Xmas Tree Collection,...,0,0,0,0,0,0,0,0,0,0
453576,ADDRESS,HPD,Department of Housing Preservation and Develop...,MANHATTAN,NaN,NaN,NaN,NEW YORK,05 MANHATTAN,HEAT/HOT WATER,...,0,0,0,0,0,0,0,0,0,0
